In [ ]:
import datetime, requests
import pandas as pd
import xlsxwriter

In [97]:
codigoINE_poblacion_provincias = {
    "Bizkaia" : 2905,
    "Gipuzkoa" : 2873,
    "Araba" : 2854
}

url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'
num_datos = 50000

In [98]:
df_poblacion = pd.DataFrame(columns=[])

for provincia in codigoINE_poblacion_provincias:
    #print(provincia)
    #print(codigoINE_poblacion_provincias[provincia])

    main_url_INE_Estructura_Datos_Poblacion = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/" + str(codigoINE_poblacion_provincias[provincia]) + "?page=1"
    INE_Estructura_Datos_Poblacion = requests.get(main_url_INE_Estructura_Datos_Poblacion).json()

    df_INE_Estructura_Datos_Poblacion = pd.DataFrame.from_dict(INE_Estructura_Datos_Poblacion)

    new = df_INE_Estructura_Datos_Poblacion["Nombre"].str.split('.')
    df_INE_Estructura_Datos_Poblacion["Nombre"] = new.map(lambda x: x[0])
    df_INE_Estructura_Datos_Poblacion["Tipo"] = new.map(lambda x: x[1])
    df_INE_Estructura_Datos_Poblacion["Tipo"] = df_INE_Estructura_Datos_Poblacion["Tipo"].str.strip()

    df_INE_Estructura_Datos_Poblacion = df_INE_Estructura_Datos_Poblacion[["Id","COD","Nombre","Tipo"]]

    df_INE_Estructura_Datos_Poblacion.drop(df_INE_Estructura_Datos_Poblacion[df_INE_Estructura_Datos_Poblacion.Nombre == "Bizkaia"].index, inplace=True)
    df_INE_Estructura_Datos_Poblacion.drop(df_INE_Estructura_Datos_Poblacion[df_INE_Estructura_Datos_Poblacion.Nombre == "Gipuzkoa"].index, inplace=True)
    df_INE_Estructura_Datos_Poblacion.drop(df_INE_Estructura_Datos_Poblacion[df_INE_Estructura_Datos_Poblacion.Nombre == "Araba/Álava"].index, inplace=True)

    #display(df_INE_Estructura_Datos_Poblacion)
    
    

    for index, poblacion in df_INE_Estructura_Datos_Poblacion.iterrows():
        codigo = poblacion["COD"]
        url_datos_poblacion = url_plantilla.format(codigo=codigo, num_datos=num_datos)
        df_poblacion_ = pd.DataFrame.from_dict(requests.get(url_datos_poblacion).json()["Data"])[["Anyo","Valor"]].rename(columns={"Anyo": "Año", "Valor": poblacion["Nombre"]})
        df_poblacion_ = df_poblacion_.set_index("Año")
        df_poblacion_.columns = pd.MultiIndex.from_product([[provincia], df_poblacion_.columns, [poblacion["Tipo"]]], names=["Provincia", 'Población', 'Tipo'])        
        df_poblacion = pd.concat([df_poblacion, df_poblacion_], axis=1)

        #display(df_poblacion_Bizkaia)
    
    

In [99]:
display(df_poblacion)

Provincia Bizkaia                                                            \
Población Abadiño                 Abanto y Ciérvana-Abanto Zierbena           
Tipo        Total Hombres Mujeres                             Total Hombres   
Año                                                                           
1996       6897.0  3469.0  3428.0                            8417.0  4176.0   
1998       6820.0  3420.0  3400.0                            8812.0  4364.0   
1999       6858.0  3440.0  3418.0                            8920.0  4395.0   
2000       6851.0  3427.0  3424.0                            8989.0  4435.0   
2001       6841.0  3423.0  3418.0                            9033.0  4479.0   
2002       6954.0  3485.0  3469.0                            9111.0  4508.0   
2003       6996.0  3509.0  3487.0                            9190.0  4550.0   
2004       7031.0  3522.0  3509.0                            9315.0  4624.0   
2005       7129.0  3595.0  3534.0                            9503.0  4717.0   
2006       7163.0  3610.0  3553.0                            9608.0  4762.0   
2007       7140.0  3594.0  3546.0                            9609.0  4738.0   
2008       7197.0  3617.0  3580.0                            9608.0  4733.0   
2009       7260.0  3647.0  3613.0                            9647.0  4779.0   
2010       7310.0  3653.0  3657.0                            9724.0  4823.0   
2011       7312.0  3631.0  3681.0                            9713.0  4830.0   
2012       7385.0  3664.0  3721.0                            9708.0  4815.0   
2013       7458.0  3697.0  3761.0                            9758.0  4813.0   
2014       7504.0  3725.0  3779.0                            9688.0  4782.0   
2015       7516.0  3745.0  3771.0                            9635.0  4754.0   
2016       7533.0  3770.0  3763.0                            9577.0  4733.0   
2017       7522.0  3748.0  3774.0                            9543.0  4702.0   
2018       7533.0  3771.0  3762.0                            9545.0  4701.0   
2019       7599.0  3815.0  3784.0                            9444.0  4668.0   
2020       7658.0  3862.0  3796.0                            9471.0  4692.0   

Provincia                                             ...     Araba           \
Población         Ajangiz                 Alonsotegi  ... Zalduondo Zambrana   
Tipo      Mujeres   Total Hombres Mujeres      Total  ...   Mujeres    Total   
Año                                                   ...                      
1996       4241.0   388.0   200.0   188.0     2894.0  ...      63.0    357.0   
1998       4448.0   401.0   203.0   198.0     2829.0  ...      61.0    366.0   
1999       4525.0   416.0   214.0   202.0     2775.0  ...      67.0    370.0   
2000       4554.0   418.0   213.0   205.0     2760.0  ...      63.0    364.0   
2001       4554.0   414.0   208.0   206.0     2703.0  ...      63.0    385.0   
2002       4603.0   425.0   217.0   208.0     2660.0  ...      66.0    375.0   
2003       4640.0   430.0   217.0   213.0     2658.0  ...      70.0    352.0   
2004       4691.0   428.0   210.0   218.0     2626.0  ...      78.0    361.0   
2005       4786.0   438.0   212.0   226.0     2745.0  ...      81.0    367.0   
2006       4846.0   441.0   217.0   224.0     2785.0  ...      86.0    362.0   
2007       4871.0   442.0   217.0   225.0     2804.0  ...      87.0    360.0   
2008       4875.0   445.0   214.0   231.0     2831.0  ...      84.0    358.0   
2009       4868.0   445.0   216.0   229.0     2835.0  ...      85.0    377.0   
2010       4901.0   450.0   225.0   225.0     2828.0  ...      87.0    383.0   
2011       4883.0   458.0   235.0   223.0     2820.0  ...      88.0    403.0   
2012       4893.0   457.0   235.0   222.0     2845.0  ...      81.0    430.0   
2013       4945.0   450.0   229.0   221.0     2827.0  ...      82.0    422.0   
2014       4906.0   463.0   239.0   224.0     2843.0  ...      83.0    417.0   
2015       4881.0   459.0  

In [103]:
display(df_poblacion["Bizkaia"]["Abadiño"]["Total"].loc[2020])

7658.0